In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

import graphviper

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.31 already installed.


## Download dataset

global (EVN+VLBA) VLBI observation with:
- 5 scans 
- 2 fields 
- 2 spw

In [2]:
graphviper.utils.data.download("global_vlbi_gg084b_reduced.ms")

[2024-07-18 14:46:33,043]  WARNING  graphviper:  File exists: /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox 
[2024-07-18 14:46:33,043]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List                  
 ─────────────────────────────── 
  global_vlbi_gg084b_reduced.ms

[2024-07-18 14:46:33,823]     INFO  graphviper:  File exists: global_vlbi_gg084b_reduced.ms 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "global_vlbi_gg084b_reduced.ms"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "global_vlbi_gg084b_reduced.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-07-18 14:46:33,980]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'FIELD_ID'] 
[2024-07-18 14:46:33,985]     INFO  graphviper:  Number of partitions: 4 
[2024-07-18 14:46:33,985]     INFO  graphviper:  DDI [0], STATE [-1], FIELD [0], SCAN [  3 319 321 552] 
[2024-07-18 14:46:34,472]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-07-18 14:46:34,975]     INFO  graphviper:  DDI [0], STATE [-1], FIELD [1], SCAN [320] 
[2024-07-18 14:46:35,235]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-07-18 14:46:35,489]     INFO  graphviper:  DDI [1], STATE [-1], FIELD [0], SCAN [  3 319 321 552] 
[2024-07-18 14:46:36,048]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-07-18 14:46:36,623]     INFO  graphviper:  DDI [1], STATE [-1], FIELD [1], SCAN [320] 
[2024-07-1

## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,obs_mode,shape,polarization,spw_id,field_id,field_name,source_id,source_name,field_coords,start_frequency,end_frequency
0,global_vlbi_gg084b_reduced_0,None,"(330, 57, 32, 4)","[RR, RL, LR, LL]",0,[0],[J1311-2329],[-1],[Unknown],"[fk5, 13h11m37.41s, -23d29m56.65s]",4.900500e+09,4.916000e+09
1,global_vlbi_gg084b_reduced_1,None,"(150, 28, 32, 4)","[RR, RL, LR, LL]",0,[1],[EM170817],[-1],[Unknown],"[fk5, 13h09m48.09s, -23d22m53.35s]",4.900500e+09,4.916000e+09
2,global_vlbi_gg084b_reduced_3,None,"(150, 28, 32, 4)","[RR, RL, LR, LL]",1,[1],[EM170817],[-1],[Unknown],"[fk5, 13h09m48.09s, -23d22m53.35s]",4.916000e+09,4.931500e+09
3,global_vlbi_gg084b_reduced_2,None,"(330, 57, 32, 4)","[RR, RL, LR, LL]",1,[0],[J1311-2329],[-1],[Unknown],"[fk5, 13h11m37.41s, -23d29m56.65s]",4.916000e+09,4.931500e+09


In [5]:
ps.keys()

dict_keys(['global_vlbi_gg084b_reduced_0', 'global_vlbi_gg084b_reduced_1', 'global_vlbi_gg084b_reduced_3', 'global_vlbi_gg084b_reduced_2'])

In [6]:
ps['global_vlbi_gg084b_reduced_0']

<xarray.Dataset> Size: 32MB
Dimensions:                     (time: 330, baseline_id: 57, frequency: 32,
                                 polarization: 4, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 228B dask.array<chunksize=(57,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 228B dask.array<chunksize=(57,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 456B 0 1 2 3 ... 53 54 55 56
  * frequency                   (frequency) float64 256B 4.9e+09 ... 4.916e+09
  * polarization                (polarization) <U2 32B 'RR' 'RL' 'LR' 'LL'
  * time                        (time) float64 3kB 1.527e+09 ... 1.527e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 150kB dask.array<chunksize=(20, 57), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 2MB dask.array<chunksize=(20, 57, 1, 4), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 150kB dask.array<chunksize=(20, 57), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 451kB dask.array<chunksize=(20, 57, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 19MB dask.array<chunksize=(20, 57, 1, 4), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 10MB dask.array<chunksize=(20, 57, 1, 4), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    partition_info:  {'field_id': [0], 'field_name': ['J1311-2329'], 'obs_mod...
    pointing_xds:    <xarray.Dataset> Size: 0B\nDimensions:  ()\nData variabl...
    antenna_xds:     <xarray.Dataset> Size: 4kB\nDimensions:        (antenna_...

In [7]:
ps['global_vlbi_gg084b_reduced_0'].attrs["antenna_xds"]

<xarray.Dataset> Size: 4kB
Dimensions:        (antenna_id: 23, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 184B 0 1 2 3 4 5 6 ... 17 18 19 20 21 22
    mount          (antenna_id) <U16 1kB dask.array<chunksize=(23,), meta=np.ndarray>
    name           (antenna_id) <U2 184B dask.array<chunksize=(23,), meta=np.ndarray>
    station        (antenna_id) <U2 184B dask.array<chunksize=(23,), meta=np.ndarray>
    type           (antenna_id) <U12 1kB dask.array<chunksize=(23,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 12B 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 184B dask.array<chunksize=(23,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 552B dask.array<chunksize=(23, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 552B dask.array<chunksize=(23, 3), meta=np.ndarray>

In [8]:
ps['global_vlbi_gg084b_reduced_0'].attrs[
    "antenna_xds"
].name.values

array(['BD', 'ZC', 'HH', 'NT', 'EF', 'JB', 'IB', 'MC', 'YS', 'TR', 'WB',
       'SC', 'HN', 'GB', 'NL', 'FD', 'YY', 'OV', 'LA', 'PT', 'KP', 'BR',
       'MK'], dtype='<U2')

In [9]:
ps[
    'global_vlbi_gg084b_reduced_0'
].baseline_antenna1_id.values

array([ 0,  0,  2,  2,  2,  2,  2,  2,  2,  8,  8,  8,  8,  8,  8, 11, 11,
       11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 14, 14, 15, 15, 15, 15, 15,
       15, 15, 16, 16, 16, 16, 16, 16, 18, 18, 18, 18, 18, 19, 19, 19, 19,
       20, 20, 20, 21, 21, 22], dtype=int32)

In [10]:
ps[
    'global_vlbi_gg084b_reduced_0'
].baseline_antenna2_id.values

array([ 0,  2,  2,  8, 11, 12, 13, 14, 15,  8, 11, 12, 13, 14, 15, 11, 12,
       13, 14, 15, 12, 13, 14, 15, 13, 14, 15, 14, 15, 15, 16, 18, 19, 20,
       21, 22, 16, 18, 19, 20, 21, 22, 18, 19, 20, 21, 22, 19, 20, 21, 22,
       20, 21, 22, 21, 22, 22], dtype=int32)